In [48]:
import numpy as np
import cv2 

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [49]:
from keras.models import load_model
from keras.models import Model

model = load_model('emotion_detection_model.h5')

In [50]:
def get_emotion(em_list):
    i = 0
    maxim = -1
    maxim_row = -1
    for emotion in em_list:
        if emotion > maxim:
            maxim = emotion
            maxim_row = i
        i = i + 1
    return maxim_row

In [62]:
import urllib
import numpy as np
import time
import ssl

from keras.preprocessing.image import img_to_array

# Replace the URL with your own IPwebcam shot.jpg IP:port
urlNeeded = 'https://192.168.1.2:8080//shot.jpg'
context = ssl._create_unverified_context()
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]

while True:
    
    with urllib.request.urlopen(urlNeeded,context=context) as url:
        s = url.read()
        imgNp=np.array(bytearray(s),dtype=np.uint8)
        img=cv2.imdecode(imgNp,-1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            crop_face = img[y:y+h, x:x+w]
            gray_crop_face = cv2.cvtColor(crop_face, cv2.COLOR_BGR2GRAY)
            gray_crop_face = cv2.resize(gray_crop_face,(48,48))
            gray_crop_face = gray_crop_face.astype("float") / 255.0
            gray_crop_face = img_to_array(gray_crop_face)
            gray_crop_face = np.expand_dims(gray_crop_face, axis=0)
            emotions = model.predict([gray_crop_face,])
            emotions = emotions.flatten()
            emotion = np.argmax(emotions)
            cv2.putText(img,EMOTIONS[emotion],(0,700), cv2.FONT_HERSHEY_SIMPLEX, 4,(100,35,200),1,cv2.LINE_AA)
        # put the image on screen
        cv2.imshow('IPWebcam',img)

    #To give the processor some less stress
    #time.sleep(0.1) 

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break